In [0]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import json
import pandas as pd
import numpy as np
import xgboost, textblob, string

In [0]:
data=[]
with open('/content/drive/My Drive/Shinra/dataset.json', 'r', encoding='utf-8') as ene_file:
    for line in ene_file.readlines():
        line = str(line).strip()
        # #workaround for escape characters
        # if "\'" in line:
        #     line = line.replace("\'", "\\'")
        data.append(json.loads(line))

In [0]:
df = pd.DataFrame(columns=['page_id','text','category'])

In [0]:
for i in range(0,30492):
    df.loc[i]=[int(data[i].get('page_id')),data[i].get('text'),int(data[i].get('classes')[0])]

In [0]:
df.astype({'category': 'int64'},copy=False).dtypes

page_id     object
text        object
category     int64
dtype: object

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30492 entries, 0 to 30491
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   page_id   30492 non-null  object
 1   text      30492 non-null  object
 2   category  30492 non-null  object
dtypes: object(3)
memory usage: 952.9+ KB


In [0]:
X=df['text'].values
y=df['category'].values

In [0]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(X, y,test_size=0.1,stratify=y)

In [0]:
count_vect = CountVectorizer()
count_vect.fit(train_x)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [0]:
xtrain_count =  count_vect.transform(train_x)
xtest_count =  count_vect.transform(test_x)

In [0]:
classifier=xgboost.XGBClassifier()

In [0]:
classifier.fit(xtrain_count.tocsc(), train_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
predictions = classifier.predict(xtest_count.tocsc())

In [0]:
preds=np.array(predictions)
preds = preds.astype('int64') 

In [0]:
test_y_n=np.array(test_y)
test_y_n= test_y_n.astype('int64') 

In [0]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y_n, preds)

0.818688524590164

In [0]:
from sklearn.metrics import classification_report
print(classification_report(test_y_n, preds))

              precision    recall  f1-score   support

           0       0.63      0.34      0.44       564
           1       0.82      0.95      0.88      2106
           2       0.98      0.93      0.95       280
           3       0.91      0.97      0.94        31
           9       0.96      0.35      0.51        69

    accuracy                           0.82      3050
   macro avg       0.86      0.71      0.74      3050
weighted avg       0.81      0.82      0.80      3050

